In [22]:
import sys
sys.path.append('..')

In [23]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline
from pathlib import Path
import numpy as np
import xgboost as xgb
import plotly.express as px
import folium
from global_variables import BRANDS, GEMSTONES, TAUX_CHANGE

from utils import get_sample_lot

In [24]:
DATA_PATH = Path('data')
VERBOSE = True

In [25]:
df = pd.read_pickle(DATA_PATH / 'certif_one_gem_processed_data.pkl')
if VERBOSE:
    display(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2592 entries, 0 to 2591
Data columns (total 29 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   lot_id             2592 non-null   int64              
 1   certifier          2592 non-null   object             
 2   certif_id          2592 non-null   object             
 3   description        2592 non-null   object             
 4   gemstone           2220 non-null   object             
 5   carat              2329 non-null   float64            
 6   color              999 non-null    object             
 7   fancy              453 non-null    object             
 8   location           923 non-null    object             
 9   clarity            1475 non-null   object             
 10  oil                166 non-null    object             
 11  heat               661 non-null    object             
 12  rounded_carat      2329 non-null   float64      

None

In [73]:
df = pd.read_pickle(DATA_PATH / 'certif_one_gem_processed_data.pkl')


In [74]:
df.columns

Index(['lot_id', 'certifier', 'certif_id', 'description', 'gemstone', 'carat',
       'color', 'fancy', 'location', 'clarity', 'oil', 'heat', 'rounded_carat',
       'number_gem', 'Title', 'auction_id', 'Description', 'EstimateLow',
       'EstimateHigh', 'PriceRealised', 'StartDate', 'EndDate', 'ImageURL',
       'EventType', 'URL', 'original_currency', 'signature', 'image_name',
       'auction_location'],
      dtype='object')

In [75]:
df = df[['lot_id', 'certifier', 'certif_id', 'description', 'gemstone', 'carat',
       'color', 'fancy', 'location', 'clarity', 'oil', 'heat', 'rounded_carat',
       'number_gem', 'Title', 'auction_id', 'Description', 'EstimateLow',
       'EstimateHigh', 'PriceRealised', 'StartDate', 'EndDate',
       'EventType','original_currency', 'signature']].to_pickle('data_st.pkl')

In [26]:
df = df[df.carat >= 1].copy()
carat_df  = df.copy().set_index('StartDate')
carat_df['price_per_ct'] = carat_df.PriceRealised / carat_df.carat

In [27]:
df.location.value_counts()

colombia      236
burma         208
ceylon        199
madagascar     23
kashmir        16
zambi          14
mozambi        10
tanzania        8
thailand        5
siam            5
tajikistan      3
malawi          2
ethiopia        1
ceylan          1
Name: location, dtype: int64

# Carat analysis

In [28]:
carat_df  = df.copy().set_index('StartDate')

In [29]:
carat_df.color.value_counts()

d    337
e    124
f    122
g     77
i     76
h     65
j     40
k     28
l     23
m     19
n     13
a     10
y     10
w      9
q      9
u      8
o      8
s      7
Name: color, dtype: int64

In [30]:
carat_df  = df.copy().set_index('StartDate')
carat_df['price_per_ct'] = carat_df.PriceRealised / carat_df.carat

In [ ]:
grouped_time_carat_price = carat_df.groupby(['gemstone']).resample('1m').agg({"price_per_ct": 'mean', 'lot_id': 'count'})

In [ ]:
grouped_time_carat_price

In [ ]:
px.bar(grouped_time_carat_price.reset_index(), x='StartDate', y='price_per_ct', color='gemstone')


In [ ]:
carat_df[carat_df.gemstone == 'diamond']#.loc['2020-11'].sort_values(by=['price_per_ct'],ascending=False)

In [ ]:
# px.line(grouped_time_carat_price.xs('sapphire', level='main_gemstone')['price_per_ct'])
get_sample_lot(carat_df, 180268)

# Diamond analysis

In [ ]:
diamond_df.columns

Index(['lot_id', 'certifier', 'certif_id', 'description', 'gemstone', 'carat',
       'color', 'fancy', 'location', 'clarity', 'oil', 'heat', 'rounded_carat',
       'number_gem', 'Title', 'auction_id', 'Description', 'EstimateLow',
       'EstimateHigh', 'PriceRealised', 'EndDate', 'ImageURL', 'EventType',
       'URL', 'original_currency', 'signature', 'image_name',
       'auction_location', 'price_per_ct'],
      dtype='object')

In [ ]:
px.bar(df.groupby('gemstone').lot_id.count())

In [ ]:
diamond_df = carat_df[carat_df.gemstone == 'diamond'].copy()

In [69]:
diamond_df.resample('1M')['PriceRealised'].sum()

StartDate
2018-11-30 00:00:00+00:00    7.383394e+07
2018-12-31 00:00:00+00:00    2.784127e+07
2019-01-31 00:00:00+00:00    0.000000e+00
2019-02-28 00:00:00+00:00    0.000000e+00
2019-03-31 00:00:00+00:00    0.000000e+00
                                 ...     
2023-07-31 00:00:00+00:00    0.000000e+00
2023-08-31 00:00:00+00:00    8.632637e+06
2023-09-30 00:00:00+00:00    2.433129e+06
2023-10-31 00:00:00+00:00    2.048458e+05
2023-11-30 00:00:00+00:00    2.731994e+07
Freq: M, Name: PriceRealised, Length: 61, dtype: float64

In [72]:
px.bar(
    diamond_df.resample("1M")["PriceRealised"].sum(),
    y="PriceRealised",
    color_discrete_sequence=px.colors.sequential.RdBu,
)

In [33]:
clarity_grouped = diamond_df.groupby("clarity").agg(
    counted=("clarity", "count"),
    valued=('PriceRealised', 'sum'),
    mean_price_per_carat=('price_per_ct', 'mean')
).reset_index()

In [34]:
fig = px.pie(
    clarity_grouped,
    values="counted",
    names="clarity",
    title="Diamond Clarity Distribution",
    color_discrete_sequence=px.colors.sequential.RdBu,
)
fig.show()

In [32]:
diamond_df.groupby('clarity').price_per_ct

In [64]:
bar_price_clarity = px.bar(
    clarity_grouped.sort_values(by="mean_price_per_carat", ascending=False),
    x="clarity",
    y="mean_price_per_carat",
    color="counted",
    color_continuous_scale=px.colors.sequential.RdBu_r,
)

bar_price_clarity.update_layout(
    title="Price of the different clarities",
    yaxis=dict(title="Average €/carat"),
    showlegend=False,
)
bar_price_clarity.update_coloraxes(showscale=False)

In [ ]:
price_moy = diamond_df.groupby('clarity').agg({'PriceRealised': 'mean'})

In [ ]:
px.box(diamond_df, x='clarity', y='PriceRealised')

# biggest sales

In [ ]:
biggest_sales = diamond_df[diamond_df.PriceRealised >=1000000].copy()

In [ ]:
get_sample_lot(biggest_sales, 217258)

In [ ]:
px.histogram(biggest_sales, x='carat')

In [ ]:
fig = px.pie(biggest_sales, names='fancy', title='Diamond Clarity Distribution')
fig.show()

# Unsaled

In [ ]:
unsold_df = diamond_df[diamond_df.PriceRealised.isna()].copy()

In [ ]:
fig = px.pie(unsold_df, names='fancy', title='Diamond Clarity Distribution')
fig.show()

# Location analysis

In [ ]:
import folium
from folium.plugins import MarkerCluster
import pandas as pd

# Clean up and process the data
df['location'] = df['location'].replace({'ceylon': 'ceylon', 'ceylan': 'ceylon', 'siam': 'thailand', 'mozambi': 'mozambique'})

name_mapping = {
    'colombia': 'Colombia',
    'burma': 'Myanmar',
    'tajikistan': 'Tajikistan',
    'ceylon': 'Sri Lanka',
    'madagascar': 'Madagascar',
    'tanzania': 'Tanzania',
    'ethiopia': 'Ethiopia',
    'zambi': 'Zambia',
    'kashmir': 'Kashmir',  # Note: Kashmir might not be a country in the GeoDataFrame
    'thailand': 'Thailand',
    'siam': 'Thailand',  # Siam is an old name for Thailand
    'mozambi': 'Mozambique',
    'malawi': 'Malawi'
}

# Count occurrences of each location
location_counts = df['location'].value_counts()

In [ ]:
df.location = df.location.map(name_mapping)

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from collections import Counter

# Count occurrences of each country
country_counts = Counter(df['location'])

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from collections import Counter

# Count occurrences of each country
country_counts = Counter(df['location'])

# Create a GeoDataFrame with world map data
world = gpd.read_file(DATA_PATH/'ne_110m_admin_0_countries/ne_110m_admin_0_countries.shp')


In [ ]:

# Update country colors based on occurrences
world['color'] = world['NAME'].apply(lambda x: country_counts.get(x, 0))




In [ ]:
sub_world = world[world.color > 0][['NAME', 'color', 'geometry']].copy()

In [ ]:
sub_world

In [ ]:
# Create a base map
m = folium.Map(location=[0, 0], zoom_start=2)

# Create a MarkerCluster layer for better visualization of markers
marker_cluster = MarkerCluster().add_to(m)

# Add markers for each country
for country, count in country_counts.items():
    country_name = name_mapping.get(country, '')
    if country_name:
        folium.Marker(location=[0, 0], popup=f"{country_name}: {count}", tooltip=country_name).add_to(marker_cluster)


In [ ]:
import matplotlib
colname = 'color'
xmin, ymin, xmax, ymax = sub_world.total_bounds

centroidx = np.mean([xmin, xmax])
centroidy = np.mean([ymin, ymax])

map1 = folium.Map(
    location=[centroidy, centroidx],
    tiles='cartodbpositron',
    zoom_start=6,
)

cmap = matplotlib.cm.get_cmap('viridis')

vmin = sub_world[colname].min()
vmax = sub_world[colname].max()


norm = matplotlib.colors.SymLogNorm(vmin=vmin, vmax=vmax, linthresh=0.1)

def fetchHexFromValue(value):
  NormedValue = norm(value)
  RGBAValue = cmap(NormedValue)
  HEXValue = matplotlib.colors.to_hex(RGBAValue)
  return HEXValue



for idx, r in sub_world.iterrows():

    lat = r["geometry"].centroid.y
    lon = r["geometry"].centroid.x
    folium.Marker(location=[lat, lon],
                  popup='idx:{0} <br> {1}: {2}'.format(idx,
                                                       colname, 
                                                       r[colname])
    ).add_to(map1)

sub_world.explore(colname, cmap="viridis", m=map1)

map1

In [ ]:
def get_color(x):
    # Normalize the value within the desired range (100 to 1300)
    normalized_value = (x - 1) / (236 - 1)

    # Map the normalized value to the red color scale (0 to 255)
    red = int(255 * normalized_value)

    # Create the RGBA color string
    rgba_color = f"rgba({red}, 0, 0)"

    return rgba_color

In [ ]:
m = folium.Map()
for _, r in sub_world.iterrows():
    # Without simplifying the representation of each borough,
    # the map might not be displayed
    sim_geo = gpd.GeoSeries(r["geometry"]).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    color = get_color(r['color'])
    print(color)
    geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {"fillColor": color})
    folium.Popup('{0}: <br> Gemstones: {1}'.format(r['NAME'], 
                                                       r[colname])).add_to(geo_j)
    geo_j.add_to(m)

In [ ]:
m

In [ ]:
# Create a folium map
m = folium.Map(location=[0, 0], zoom_start=2)

# Add GeoJson layer with custom styling
folium.GeoJson(
    data=sub_world,
    name='geojson',
    style_function=lambda x: {"fillColor": f"rgba({x['properties']['color']}, 0, 0, 0)"}
).add_to(m)

In [ ]:
m

In [ ]:
m

# Drafts

https://onlineonly.christies.com/s/jewels-online-iconic-designs/graff-diamond-eternity-band-ring-19/102322?ldp_breadcrumb=back


In [ ]:
grouped_time_carat_price.xs('sapphire', level='main_gemstone')